# GENERATIVE ADVERSARIAL NETWORK (GAN) PYTORCH
In this tutorial a GAN is implemented using Pytorch. A GAN basically uses a decoder in order to generate images, which at the beginning will make no sense since the latent variables are sampled from random noise. Then there is a neural network that acts as a discriminator which function is basically to identify real images and discard those which are fake. The generator and the discriminator will start a challenge in a way that the generator will try to fool the discriminator generating images that it cannot predict wether they are real or fake. This can be done thanks to the loss function:
The discriminator loss function will have to parts, the error comming from real images which will try to minimize the difference between the predicted targets and a tensor with all ones (all ones mean that they are all real), and the error comming from the generator (fake images) which will try to minimize the difference between a tensor with all zeros (all zeros mean they are fake) and the images generated with the generator which should be considered fake by the discriminator. 
The generator loss function has to minimize the difference between the output of the discriminator when it has been fed with fake that and a tensor of all ones (meaning all real). This way the minmax game competition starts!
In all the pictures there are some transformations applied before they are fed to the neural net (data preprocessing). Other types of preprocessing functions can be pre-defined such as flippers or zoomers in order to perform data augmentation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data 
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os


""" PARAMETERS """

BATCH_SIZE = 64
IMAGE_SIZE = 64
PATH_IMAGES = "" #The training images directory
PATH = ""  #path to save the images generated
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("using device: ", device)
NUMBER_OF_EPOCHS = 100

def preprocess():

    #Image transformations (applying different transformations)
    transform = transforms.Compose([transforms.Scale(IMAGE_SIZE),
    transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),])

    # Loading the dataset
    #dataset = dset.CIFAR10(root=path, download = True, transform = transform)
    dataset = dset.ImageFolder(root=PATH_IMAGES, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)
    
    return dataloader

        
class DiscriminatorNet(torch.nn.Module):

    def __init__(self):
        super(DiscriminatorNet, self).__init__() #Initializing superclass
        self.model = nn.Sequential(
            
            #In pytorch out_channels equals number of filters applied
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1, bias = False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1, kernel_size=4, stride=1, padding=0, bias = False),
            nn.Sigmoid()
            )
        
        
    def forward(self, input):

        output = self.model(input)
        return output.view(-1) #View method reshapes!
    
    
    
class GeneratorNet(torch.nn.Module):

    def __init__(self):
        super(GeneratorNet, self).__init__()

        self.model = nn.Sequential(

            nn.ConvTranspose2d(in_channels=100, out_channels=512, kernel_size=4, stride=1, padding=0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh() #The output is squased from -1 to 1 using the parabolic tangent function

            )
        
    def forward(self, input):

        output = self.model(input)
        return output
    
def optimizer():
    
    discriminator = DiscriminatorNet()
    discriminator.cuda()
    generator = GeneratorNet()
    generator.cuda()
    criterion = nn.BCELoss()
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    return discriminator, generator, criterion, optimizer_d, optimizer_g

def train():
    
    discriminator, generator, criterion, optimizer_d, optimizer_g = optimizer()
    dataloader = preprocess()
    for epoch in range(NUMBER_OF_EPOCHS):

        for i, data in enumerate(dataloader,0):

            #Now we train the discrimnator which are the real data and the classifier label for
            #real data is one, this is why the target is built using a torch.ones which builds a matrix
            #in which all the values equals one, and of course the size of the squared-matrix is the input size!

            discriminator.zero_grad() #Gradients set to zero!
            real, _ = data
            inp = Variable(real)
            target = Variable(torch.ones(inp.size()[0]))
            output = discriminator(inp.cuda())
            errD_real = criterion(output.cuda(), target.cuda())


            # NOw we train the discriminator with the fake data which labels are 0 and thus, we use the torch.zeros function!

            noise = Variable(torch.randn(inp.size()[0], 100, 1, 1))
            print("latent variable ", noise)
            fake = generator(noise.cuda())
            target = Variable(torch.zeros(inp.size()[0]))
            output = discriminator(fake.detach()) #Using descriminator with the fake that comes from the generator
            errD_fake = criterion(output.cuda(), target.cuda()) #Minimizing the different between all zeros (False) 
            #and the images produced by the generator that should be all zero as well.

            errD = errD_real + errD_fake
            errD.backward()
            optimizer_d.step()


            #NOW THE GENERATOR IS TRAINED!!

            generator.zero_grad()
            target = Variable(torch.ones(inp.size()[0]))
            output = discriminator(fake.cuda())
            errG = criterion(output.cuda(), target.cuda()) #Here we want the discriminator to think that the fake are ones
            #since ones mean not fake!
            errG.backward()
            optimizer_g.step()

            #print("loss_g {} loss_d {}".format(errG.data, errD.data))
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 50, i, len(dataloader), errD.data, errG.data))
            if i % 100 == 0:
                vutils.save_image(real, '%s/real_samples.png' % PATH, normalize = True)
                fake = generator(noise.cuda())
                vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % (PATH, epoch), normalize = True)
                        
                        
                        
if __name__ == "__main__":
    train()
                        
                        
                        